In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')

In [5]:
from src.dataset import *
from src.concept_vectors import *
from src.util import *
from src.hierarchy import *
from src.metrics import *
import numpy as np
import matplotlib.pyplot as plt

In [6]:
mnist_attributes = get_mnist_attributes()

In [7]:
seeds = [43,44,45]
hierarchy_creation_method = create_linkage_hierarchy
dataset = 'mnist'
attributes = mnist_attributes

In [8]:
metrics = [stability_metric,robustness_image_metric,responsiveness_image_metric]
metric_names = ['Stability', 'Image Robustness', 'Image Responsiveness']

### Evalaute Labels-Only vectors

In [11]:
for metric,name in zip(metrics,metric_names):
    score = metric(hierarchy_creation_method,
                                    load_label_vectors_simple,
                                    dataset,
                                    attributes,
                                    seeds)
    print("{}: {}".format(name, score))

Stability: 0.0
Image Robustness: 0.0
Image Responsiveness: 47.0


### Evaluate TCAV Vectors

In [12]:
for metric,name in zip(metrics,metric_names):
    score = metric(hierarchy_creation_method,
                                    load_tcav_vectors_simple,
                                    dataset,
                                    attributes,
                                    seeds)
    print("{}: {}".format(name, score))

Stability: 43.0
Image Robustness: 46.666666666666664
Image Responsiveness: 46.0
